# Free Attempt for the DMML Project, Ahmed and Joris
Inspired by: https://www.kaggle.com/code/houssemayed/camembert-for-french-tweets-classification/notebook

We will use CamemBERT to do the classification. Note: You will need to use a GPU on colab to run this computationally heavy model!

First, load packages and data

In [4]:
!pip install transformers
!pip install sentencepiece

# read data and import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")

!pip install -U spacy

# import some additional packages
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn. preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import torch.nn as nn
import torch
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# train data
df_train = pd.read_csv('training_data.csv')

# Rename Labeling
df_train['difficulty'] = df_train['difficulty'].replace(['A1','A2','B1', 'B2', 'C1', 'C2'],[0,1,2,3,4,5])

# test data
df_test = pd.read_csv('unlabelled_test_data.csv')


# Imports to use a pipeline
from tqdm import tqdm, trange
from tensorflow import keras 

# Imports for modelling
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertModel
from transformers import BertTokenizer, BertForSequenceClassification


# Evaluate the model
def evaluate(true, pred):
    precision = precision_score(true, pred, average = 'weighted')
    recall = recall_score(true, pred, average = 'weighted')
    f1 = f1_score(true, pred, average = 'weighted')
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/809k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

...

Validation Accuracy: 0.565625
Train loss: 0.5896254610270262


Epoch:  80%|████████  | 4/5 [10:44<02:40, 160.93s/it]

Validation Accuracy: 0.5854166666666667
Train loss: 0.40823096440484125


Epoch: 100%|██████████| 5/5 [13:25<00:00, 161.04s/it]

Validation Accuracy: 0.5739583333333333


# Set Parameters, load pretrained Tokenizer

In [ ]:
# These parameters are not tuned, but we checked a few possiblities to get close to the ideal value
epochs = 5
MAX_LEN = 64
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Here we load the CamemBERT tokenizer and use lower_case
tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-large', do_lower_case=True)

# As list is required as input
text = df_train['sentence'].to_list()
labels = df_train['difficulty'].to_list()


# Preparing Input for the CamemBERT model


In [1]:
# CamamBERT tokenizer is used to transform input_ids'
input_ids  = [tokenizer.encode(sent,add_special_tokens=True,max_length=MAX_LEN) for sent in text]

# the input tokens are padded
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []

# Use 0s for padding and creating a mask of 1s
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)


# Split 80/20 Train/Validation
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.2)


# torch tensors is the required datatype for the BERT model, so change input
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Create an iterator, as it will use less memory than a loop
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


NameError: ignored

# Load and prepare CamemBERT model



In [ ]:
# Load model and set it to device
model = CamembertForSequenceClassification.from_pretrained("camembert/camembert-large", num_labels=6)
model.to(device)

# Parameter settings
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# Use AdamW as optimizer
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, eps=1e-8)

# Calculate the accuracy, simply assign the text difficulty to the one with the highest probability (argmax)
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Linear scheduler for warmup
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_labels)*epochs)

# Store loss
train_loss_set = []


# Train already pre-trained model

In [ ]:
# for loop over each epoch (5)
for _ in trange(epochs, desc="Epoch"):  

    # Keep track of variables during the training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Starting trained the pre-trained BERT model on our data set
    model.train()
    for step, batch in enumerate(train_dataloader):

        # Assign batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Optain inputs from the previously made dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # Obtain loss value
        loss = outputs[0]
        # Append to training loss set
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Step optimizer (gradient) and scheduler
        optimizer.step()
        scheduler.step()

        # Update variables that track results
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    

    # Check the results on the validation set
    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Put model in validation mode
    model.eval()
    # Evaluation results for one epoch
    for batch in validation_dataloader:
        # Batch assigned to GPU
        batch = tuple(t.to(device) for t in batch)
        # unpack inputs
        b_input_ids, b_input_mask, b_labels = batch
        # don't compute/store gradients
        with torch.no_grad():
            # Forward pass, and calculate predictions
            outputs =  model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    # Print resulting validation accuracy
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))


# Check performance on Validation set

In [5]:
# Evaluate on validation set
eval_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(validation_inputs.to(device),token_type_ids=None, attention_mask=validation_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    eval_pred.extend(np.argmax(logits, axis=1).flatten())

evaluate(validation_labels, eval_pred)

CONFUSION MATRIX:
[[139  26   1   0   0   0]
 [ 41  75  39   1   2   0]
 [ 24  45  86   9   1   1]
 [  0   4  32  85  26   6]
 [  0   0   8  42  79  23]
 [  0   1   3  37  37  87]]
ACCURACY SCORE:
0.5740
CLASSIFICATION REPORT:
	Precision: 0.5795
	Recall: 0.5740
	F1_Score: 0.5717


# After Training the model, make predictions

Use the format from the sample submission.

In [ ]:
comments = df_test['sentence'].to_list()

# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in comments]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

# Apply the finetuned model (Camembert)
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())

for i in range(len(flat_pred)):
    print('Comment: ', comments[i])
    print('Label', flat_pred[i])

df_samplesub = pd.read_csv('sample_submission.csv')
df_samplesub['difficulty'] = flat_pred

# Translate back
df_samplesub['difficulty'] = df_samplesub['difficulty'].replace([0,1,2,3,4,5],['A1','A2','B1', 'B2', 'C1', 'C2'])

print(df_samplesub)

# Upload pandas dataframe as csv to drive
from google.colab import drive
drive.mount('drive')

df_samplesub.to_csv('FinalTest4.csv', index = False)
!cp FinalTest4.csv "drive/My Drive/"


Comment:  Nous dûmes nous excuser des propos que nous eûmes prononcés
Label 5
Comment:  Vous ne pouvez pas savoir le plaisir que j'ai de recevoir cette bonne nouvelle.
Label 1
Comment:  Et, paradoxalement, boire froid n'est pas la bonne parade.
Label 2
Comment:  Ce n'est pas étonnant, car c'est une saison mystérieuse
Label 2
Comment:  Le corps de Golo lui-même, d'une essence aussi surnaturelle que celui de sa monture, s'arrangeait de tout obstacle matériel, de tout objet gênant qu'il rencontrait en le prenant comme ossature et en se le rendant intérieur, fût-ce le bouton de la porte sur lequel s'adaptait aussitôt et surnageait invinciblement sa robe rouge ou sa figure pâle toujours aussi noble et aussi mélancolique, mais qui ne laissait paraître aucun trouble de cette transvertébration.
Label 5
Comment:  Elle jeta un cri, un petit cri, voulut se dresser, se débattre, le repousser ; puis elle céda, comme si la force lui eût manqué pour résister plus longtemps.
Label 4
Comment:  Madame, 